In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
#载入数据库
mnist = input_data.read_data_sets("MNIST_data/",one_hot= True)


# n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x,[-1,28,28,1])

W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 14*14*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)

#此时连接推测是rnn维度需要二维 此时 h_pool2




# 第一个全连接层
W_fc1 = weight_variable([7*7*64,784])
b_fc1 = bias_vairable([784])
# 7*7*64的图像变成1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# print("咋回事怎么变",h_fc1)

# # 第二个全连接层
# W_fc2 = weight_variable([1024,10])
# b_fc2 = bias_vairable([10])
# logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2












#初始化权重
weights = tf.Variable(tf.truncated_normal([100,10],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[10]))

def RUN(X,weights,biases):
    # inputs = [batch_size,max_time,n_inputs]
#     print('2',X)
    inputs = tf.reshape(X,[-1,28,28])
    #定义LSTM基本CELL
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32,time_major = False)
#     print('3',final_state[1])
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results

#计算RUN的返回结果
prediction = RUN(h_fc1,weights,biases)
# print(3333)
# print(prediction)
# print(555555555)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver = tf.train.Saver()
#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start = datetime.now()
    for epoch in range(10001):

        batch_xs,batch_ys =mnist.train.next_batch(50)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))

    end = datetime.now()
    latency = (end - start).total_seconds()
    print('训练10000步总时长为 '+str(latency)+' 秒 ')
    saver.save(sess,"E:/www/tf/crnn/a_model.ckpt")


